In [ ]:
# pip install neo4j yfiles_jupyter_graphs graphdatascience

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

class PersonaGraphDB:
    def __init__(self):
        load_dotenv()
        uri = os.getenv("NEO4J_URI")
        user = os.getenv("NEO4J_USER")
        password = os.getenv("NEO4J_PASSWORD")
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def clear_database(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

    def create_user_node(self, user_id):
        query = "MERGE (:User {id: $user_id})"
        with self.driver.session() as session:
            session.run(query, user_id=user_id)

    def create_conversation_node(self, conv_id, user_id):
        query = """
        MERGE (c:Conversation {id: $conv_id})
        WITH c
        MATCH (u:User {id: $user_id})
        MERGE (u)-[:STARTED]->(c)
        """
        with self.driver.session() as session:
            session.run(query, user_id=user_id, conv_id=conv_id)

    def insert_persona_fact(self, user_id, conv_id, triplet, topic):
        _, relation, obj = triplet  # subject is ignored

        query = """
        // Ensure object node
        MERGE (o:Object {name: $object})
        
        // Ensure topic and its relation to object
        MERGE (t:Topic {name: $topic})
        MERGE (t)-[:RELATION {type: $relation}]->(o)
        
        // Ensure conversation node and link to topic
        MERGE (c:Conversation {id: $conv_id})
        MERGE (c)-[:MENTIONED]->(t)
        
        // PASS conversation forward to MATCH the user and connect
        WITH c
        MATCH (u:User {id: $user_id})
        MERGE (u)-[:STARTED]->(c)
        """

        with self.driver.session() as session:
            session.run(
                query,
                object=obj,
                relation=relation,
                topic=topic,
                conv_id=conv_id,
                user_id=user_id
            )

In [5]:
db = PersonaGraphDB()
db.clear_database()

user_id = "salehaf"
conv_id = "conv001"
triplet = ("i", "like_watching", "horror movies")
topic = "movies"

db.create_user_node(user_id)
db.create_conversation_node(conv_id, user_id)
db.insert_persona_fact(user_id, conv_id, triplet, topic)

db.close()
print("✅ Triplet inserted successfully.")


✅ Triplet inserted successfully.


In [ ]:
# pip install pyvis

In [ ]:
from pyvis.network import Network
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from IPython.display import display, HTML

# Connect to Neo4j
load_dotenv()
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")
driver = GraphDatabase.driver(uri, auth=(user, password))

# Create PyVis network
# net = Network(notebook=True, directed=True)
net = Network(notebook=True, directed=True, cdn_resources="in_line")

net.force_atlas_2based()

def fetch_and_visualize(user_id):
    with driver.session() as session:
        query = """
        MATCH (u:User {id: $user_id})-[:STARTED]->(c:Conversation)-[:MENTIONED]->(t:Topic)-[r:RELATION]->(o:Object)
        RETURN u.id AS user, c.id AS conv, t.name AS topic, r.type AS rel, o.name AS object
        """
        results = session.run(query, user_id=user_id)

        # Add nodes and edges
        for record in results:
            user = record["user"]
            conv = record["conv"]
            topic = record["topic"]
            rel = record["rel"]
            obj = record["object"]

            net.add_node(user, label=user, color="#8dd3c7", shape="ellipse")
            net.add_node(conv, label=conv, color="#fdb462", shape="box")
            net.add_node(topic, label=topic, color="#bebada", shape="ellipse")
            net.add_node(obj, label=obj, color="#fb8072", shape="dot")

            net.add_edge(user, conv, label="STARTED")
            net.add_edge(conv, topic, label="MENTIONED")
            net.add_edge(topic, obj, label=rel)

    return net

# Display graph
user_id = "salehaf"
net = fetch_and_visualize(user_id)
net.show("src/frontendpersona_graph.html")